# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 12 2022 3:37PM,252887,21,635923,"NBTY Global, Inc.",Released
1,Dec 12 2022 3:34PM,252886,20,8875988,"Exact-Rx, Inc.",Released
2,Dec 12 2022 3:33PM,252885,19,MSP216692,"Methapharm, Inc.",Released
3,Dec 12 2022 3:33PM,252884,10,Clear-8894345,"ClearSpec, LLC",Released
4,Dec 12 2022 3:32PM,252883,16,8893788,Sartorius Bioprocess Solutions,Released
5,Dec 12 2022 2:55PM,252882,10,CTF0011536,"Citieffe, Inc.",Executing
6,Dec 12 2022 2:55PM,252882,10,CTF0011537,"Citieffe, Inc.",Executing
7,Dec 12 2022 2:55PM,252882,10,CTF0011538,"Citieffe, Inc.",Released
8,Dec 12 2022 2:42PM,252880,19,ADV80008963,"AdvaGen Pharma, Ltd",Released
9,Dec 12 2022 2:42PM,252880,19,ADV80008964,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,252883,Released,1
40,252884,Released,1
41,252885,Released,1
42,252886,Released,1
43,252887,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
252883,NaN,1.0
252884,NaN,1.0
252885,NaN,1.0
252886,NaN,1.0
252887,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
252778,0.0,1.0
252783,1.0,0.0
252784,1.0,0.0
252785,0.0,8.0
252793,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
252778,0,1
252783,1,0
252784,1,0
252785,0,8
252793,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,252778,0,1
1,252783,1,0
2,252784,1,0
3,252785,0,8
4,252793,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,252778,,1
1,252783,1,
2,252784,1,
3,252785,,8
4,252793,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 12 2022 3:37PM,252887,21,"NBTY Global, Inc."
1,Dec 12 2022 3:34PM,252886,20,"Exact-Rx, Inc."
2,Dec 12 2022 3:33PM,252885,19,"Methapharm, Inc."
3,Dec 12 2022 3:33PM,252884,10,"ClearSpec, LLC"
4,Dec 12 2022 3:32PM,252883,16,Sartorius Bioprocess Solutions
5,Dec 12 2022 2:55PM,252882,10,"Citieffe, Inc."
8,Dec 12 2022 2:42PM,252880,19,"AdvaGen Pharma, Ltd"
25,Dec 12 2022 2:36PM,252879,19,"GUSA Granules USA, Inc."
26,Dec 12 2022 2:33PM,252877,10,"Senseonics, Incorporated"
27,Dec 12 2022 2:22PM,252872,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 12 2022 3:37PM,252887,21,"NBTY Global, Inc.",,1
1,Dec 12 2022 3:34PM,252886,20,"Exact-Rx, Inc.",,1
2,Dec 12 2022 3:33PM,252885,19,"Methapharm, Inc.",,1
3,Dec 12 2022 3:33PM,252884,10,"ClearSpec, LLC",,1
4,Dec 12 2022 3:32PM,252883,16,Sartorius Bioprocess Solutions,,1
5,Dec 12 2022 2:55PM,252882,10,"Citieffe, Inc.",2,1
6,Dec 12 2022 2:42PM,252880,19,"AdvaGen Pharma, Ltd",6,11
7,Dec 12 2022 2:36PM,252879,19,"GUSA Granules USA, Inc.",,1
8,Dec 12 2022 2:33PM,252877,10,"Senseonics, Incorporated",,1
9,Dec 12 2022 2:22PM,252872,19,"NAPP Technologies, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 3:37PM,252887,21,"NBTY Global, Inc.",1,
1,Dec 12 2022 3:34PM,252886,20,"Exact-Rx, Inc.",1,
2,Dec 12 2022 3:33PM,252885,19,"Methapharm, Inc.",1,
3,Dec 12 2022 3:33PM,252884,10,"ClearSpec, LLC",1,
4,Dec 12 2022 3:32PM,252883,16,Sartorius Bioprocess Solutions,1,
5,Dec 12 2022 2:55PM,252882,10,"Citieffe, Inc.",1,2
6,Dec 12 2022 2:42PM,252880,19,"AdvaGen Pharma, Ltd",11,6
7,Dec 12 2022 2:36PM,252879,19,"GUSA Granules USA, Inc.",1,
8,Dec 12 2022 2:33PM,252877,10,"Senseonics, Incorporated",1,
9,Dec 12 2022 2:22PM,252872,19,"NAPP Technologies, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 3:37PM,252887,21,"NBTY Global, Inc.",1,
1,Dec 12 2022 3:34PM,252886,20,"Exact-Rx, Inc.",1,
2,Dec 12 2022 3:33PM,252885,19,"Methapharm, Inc.",1,
3,Dec 12 2022 3:33PM,252884,10,"ClearSpec, LLC",1,
4,Dec 12 2022 3:32PM,252883,16,Sartorius Bioprocess Solutions,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 3:37PM,252887,21,"NBTY Global, Inc.",1.0,NaN
1,Dec 12 2022 3:34PM,252886,20,"Exact-Rx, Inc.",1.0,NaN
2,Dec 12 2022 3:33PM,252885,19,"Methapharm, Inc.",1.0,NaN
3,Dec 12 2022 3:33PM,252884,10,"ClearSpec, LLC",1.0,NaN
4,Dec 12 2022 3:32PM,252883,16,Sartorius Bioprocess Solutions,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 3:37PM,252887,21,"NBTY Global, Inc.",1.0,0.0
1,Dec 12 2022 3:34PM,252886,20,"Exact-Rx, Inc.",1.0,0.0
2,Dec 12 2022 3:33PM,252885,19,"Methapharm, Inc.",1.0,0.0
3,Dec 12 2022 3:33PM,252884,10,"ClearSpec, LLC",1.0,0.0
4,Dec 12 2022 3:32PM,252883,16,Sartorius Bioprocess Solutions,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4551012,184.0,14.0
12,505640,1.0,1.0
16,505719,2.0,0.0
19,2528542,18.0,12.0
20,758526,24.0,2.0
21,1011372,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4551012,184.0,14.0
1,12,505640,1.0,1.0
2,16,505719,2.0,0.0
3,19,2528542,18.0,12.0
4,20,758526,24.0,2.0
5,21,1011372,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,184.0,14.0
1,12,1.0,1.0
2,16,2.0,0.0
3,19,18.0,12.0
4,20,24.0,2.0
5,21,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,184.0
1,12,Released,1.0
2,16,Released,2.0
3,19,Released,18.0
4,20,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,14.0,1.0,0.0,12.0,2.0,0.0
Released,184.0,1.0,2.0,18.0,24.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,14.0,1.0,0.0,12.0,2.0,0.0
1,Released,184.0,1.0,2.0,18.0,24.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,14.0,1.0,0.0,12.0,2.0,0.0
1,Released,184.0,1.0,2.0,18.0,24.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()